# define all locations, HRRR Variables, and time range, then fetch data

In [1]:
from pathlib import Path
from datetime import datetime

from pyhrrrzarr.hrrr.schema import HRRRVariable, VariableName, Level
from pyhrrrzarr.hrrrzarr.requests import create_requests, csv_to_locations


csv_path = Path("../data/WiscoNet pre-2024 stations.csv").resolve()
locations = csv_to_locations(csv_path)
vars = [
    HRRRVariable(name=VariableName.TMP, level=Level._2M_ABOVE_GROUND),
    HRRRVariable(name=VariableName.DPT, level=Level._2M_ABOVE_GROUND),
    # HRRRVariable(name=VariableName.PRATE, level=Level._SURFACE),
    HRRRVariable(name=VariableName.RH, level=Level._2M_ABOVE_GROUND),
    HRRRVariable(name=VariableName.UGRD, level=Level._10M_ABOVE_GROUND),
    HRRRVariable(name=VariableName.VGRD, level=Level._10M_ABOVE_GROUND),
]
level_type = "sfc"
model_type = "anl"
start=datetime(2024, 1, 1, 0, 0)
# end=datetime(2024, 1, 2, 0, 0)
end=datetime.now()

requests = create_requests(
    locations=locations,
    variables=vars,
    level_type=level_type,
    model_type=model_type,
    start=start,
    end=end
)

print(f"created {len(requests)} requests")

/Users/imcconnell/PycharmProjects/pyhrrrzarr/.pixi/envs/default/lib/python3.13/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_type" in ZarrId has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


created 517440 requests


In [ ]:
from pyhrrrzarr.hrrrzarr.fetch import get_all_request_values

await get_all_request_values(requests)


/Users/imcconnell/PycharmProjects/pyhrrrzarr/.pixi/envs/default/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching HRRR data:   7%|▋         | 35500/517440 [14:05<3:11:51, 41.87it/s]

In [ ]:
from pyhrrrzarr.hrrrzarr.requests import requests_to_df

df = requests_to_df(requests)
df.tail()

,location,lat,lon,run_hour,var_name,var_level,value,units
2011,OJNR,43.025,-89.54,2024-01-01 23:00:00,VGRD,10m_above_ground,2.455078,m/s
2012,RNLD,45.655,-89.27,2024-01-01 23:00:00,VGRD,10m_above_ground,1.579102,m/s
2013,SODO,44.779,-87.36,2024-01-01 23:00:00,VGRD,10m_above_ground,1.829102,m/s
2014,SPNR,45.827,-91.86,2024-01-01 23:00:00,VGRD,10m_above_ground,3.205078,m/s
2015,WCRS,44.185,-90.74,2024-01-01 23:00:00,VGRD,10m_above_ground,1.829102,m/s


In [ ]:
from pyhrrrzarr.hrrrzarr.requests import add_wind_speed_and_direction

df = add_wind_speed_and_direction(df)
df.tail()
    



,location,lat,lon,run_hour,var_name,var_level,value,units
2683,OJNR,43.025,-89.54,2024-01-01 23:00:00,WIND_DIR,10m_above_ground,236.218599,degrees
2684,RNLD,45.655,-89.27,2024-01-01 23:00:00,WIND_DIR,10m_above_ground,236.873897,degrees
2685,SODO,44.779,-87.36,2024-01-01 23:00:00,WIND_DIR,10m_above_ground,243.113293,degrees
2686,SPNR,45.827,-91.86,2024-01-01 23:00:00,WIND_DIR,10m_above_ground,207.520608,degrees
2687,WCRS,44.185,-90.74,2024-01-01 23:00:00,WIND_DIR,10m_above_ground,228.188670,degrees


In [ ]:
df.to_parquet("wisco.parquet")

# precip

In [ ]:
# Let's grab an analysis file using s3fs
import s3fs
import xarray as xr


fs = s3fs.S3FileSystem(anon=True)
# url = "hrrrzarr/sfc/20240101/20240101_00z_anl.zarr/surface/PRATE/surface/PRATE"
url = "hrrrzarr/sfc/20200903/20200903_00z_fcst.zarr/2m_above_ground/TMP"
store = s3fs.S3Map(root=url, s3=fs, check=False)
ds = xr.open_zarr(store)
ds

<xarray.Dataset> Size: 23kB
Dimensions:                  (time: 36, projection_x_coordinate: 1799,
                              projection_y_coordinate: 1059)
Coordinates:
  * projection_x_coordinate  (projection_x_coordinate) float64 14kB -2.698e+0...
  * projection_y_coordinate  (projection_y_coordinate) float64 8kB -1.587e+06...
  * time                     (time) datetime64[ns] 288B 2020-09-03T01:00:00 ....
Data variables:
    forecast_period          (time) timedelta64[ns] 288B ...
    forecast_reference_time  datetime64[ns] 8B ...